In [22]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split


In [23]:
data = pd.read_csv("sample_data.csv")  # Replace with your actual CSV file name
print(data.columns)

Index(['User ID', 'Age', 'Diseases', 'Allergies', 'Product ID', 'Name',
       'Brand', 'Ingredients', 'Health Benefits', 'Potential Risks',
       'Recommendation Score'],
      dtype='object')


In [39]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer

# Load data
data = pd.read_csv('sample_data.csv')

# Preprocessing
for column in ['Diseases', 'Allergies']:
    # Split the string on the comma to create a list of items
    data[column] = data[column].apply(lambda x: x.split(', ') if pd.notna(x) else [])

    # Use MultiLabelBinarizer to convert the list into a binary matrix
    mlb = MultiLabelBinarizer()
    expandedLabelData = mlb.fit_transform(data[column])
    labelClasses = mlb.classes_

    # Create a pandas DataFrame from our numpy array
    expandedLabels = pd.DataFrame(expandedLabelData, columns=labelClasses)

    # Concatenate the original DataFrame and the new one
    data = pd.concat([data, expandedLabels], axis=1)

# Continue with the rest of the steps as before

In [52]:
# Convert list-like structures to strings
for column in data.columns:
    data[column] = data[column].astype(str)

# Now you can use get_dummies
data_encoded = pd.get_dummies(data)

In [53]:
Y=data['target']
x=data_encoded
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [59]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Get the union of all columns in training and test set
all_columns = pd.concat([X_train, X_test]).columns

# Apply get_dummies and reindex with all_columns to ensure the same columns in both dataframes
X_train_encoded = pd.get_dummies(X_train).reindex(columns=all_columns, fill_value=0)
X_test_encoded = pd.get_dummies(X_test).reindex(columns=all_columns, fill_value=0)

# Continue with your code...

clf = RandomForestClassifier(random_state=42)

# Train the model
clf.fit(X_train_encoded, y_train)

# Make predictions
y_pred = clf.predict(X_test_encoded)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Model accuracy: {accuracy}')

Model accuracy: 0.0
